## INTERVIEW

In [3]:
import pandas as pd

In [19]:
!ls ../../dissertation_OLD2

 BOOKS-01-Reading_books_from_pdf_and_saving.ipynb   SantaBarbara-01.ipynb
 Buckeye-00-Cleaning.ipynb			   'Untitled Folder'
 Buckeye-01-Runnig_Perspective.ipynb		    config
 Buckeye-02-Perspective-Analysis.ipynb		    data
 CORAL-01-reading_sampling_scoring.ipynb	    notebooks
 CORAL-02-analysing_ploting.ipynb		    utils


In [17]:
pd.read_csv('../../INTERVIEW/host-map.csv')


FileNotFoundError: [Errno 2] No such file or directory: '../../INTERVIEW/splits-ns2.csv'

In [12]:
episodes = pd.read_csv('../../INTERVIEW/episodes.csv')
episodes


,id,program,title,episode_date
0,98814,Morning Edition,Senate Ushers In New Year With 'Fiscal Cliff' ...,2013-01-01
1,98824,Morning Edition,Cheap Bubbly Or Expensive Sparkling Wine? Look...,2012-12-31
2,98821,Morning Edition,U.S. Gas Prices Reach Record Level In 2012,2013-01-01
3,98806,Morning Edition,House Approves 'Fiscal Cliff' Measure,2013-01-02
4,98823,Morning Edition,The Fiscal Cliff: A Love Story,2012-12-31
...,...,...,...,...
105843,98839,All Things Considered,"With Malaysia Airlines Crash, A Loss For AIDS ...",2014-07-19
105844,98840,All Things Considered,"The Israeli-Palestinian Conflict, Remixed And ...",2014-07-19
105845,98842,All Things Considered,"As Superheroes Go Mainstream, Comic Cons Get C...",2014-07-19
105846,98833,All Things Considered,"Despite California's Drought, Taps Still Flowi...",2014-07-20


In [9]:
interview = pd.read_csv('../../INTERVIEW/utterances-2sp.csv')


In [11]:
interview

,episode,episode_order,turn_order,speaker_order,host_id,is_host,utterance
0,1,1,0,0,0,True,The impeachment inquiry picks up tomorrow wher...
1,1,1,1,0,0,True,"Just this morning, the lawyer for the whistleb..."
2,1,1,2,0,0,True,There's are a lot of moving parts.
3,1,1,3,0,0,True,"Fortunately, NPR's Mara Liasson is here to help."
4,1,2,0,0,0,True,Good morning.
...,...,...,...,...,...,...,...
1240107,141179,21,1,1,-1,False,It's an interesting city and it's so charming ...
1240108,141179,22,0,0,34,False,"Thank you, Ms. Deluca."
1240109,141179,23,0,1,-1,False,Thank you so much.
1240110,141179,24,0,0,34,False,Isabelle Deluca.


In [3]:
interview = pd.read_csv('../INTERVIEW/utterances-2sp.csv')
interview.head()

FileNotFoundError: [Errno 2] No such file or directory: '../INTERVIEW/utterances-2sp.csv'

## Sampling

In [16]:
SAMPLE_SZ = 10000
SAMPLE_SZ = min(SAMPLE_SZ,len(data),len(interview))

sample_interview = interview[interview.is_host==False].utterance.sample(SAMPLE_SZ)
sample_coraal = data.sentences.sample(SAMPLE_SZ)

### Saving samples

In [18]:
!rm data/coraal/INTERVIEW/00_sampled/*

In [19]:
for item in sample_interview.iteritems():
    with open('data/coraal/INTERVIEW/00_sampled/'+str(item[0]), 'w') as f:
        f.write(item[1])
        f.close()

In [20]:
!rm data/coraal/CORAAL/00_sampled/*

In [21]:
for item in sample_coraal.iteritems():
    with open('data/coraal/CORAAL/00_sampled/'+str(item[0]), 'w') as f:
        f.write(item[1])
        f.close()

## Defining function to run Perspective

In [22]:
%%capture
!pip install --upgrade google-api-python-client

In [23]:
import yaml
import glob
import os
import pandas as pd
import time
import re
import logging
import sys

from tqdm import tqdm
from googleapiclient import discovery

sys.path.append("utils")
from toxicity_api_communication import get_toxicity_score

In [24]:
credentials = yaml.load(open("config/credentials.yaml"))["perspective-api"]

/tmp/ipykernel_7250/2281209377.py:1: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  credentials = yaml.load(open("config/credentials.yaml"))["perspective-api"]


In [25]:
def calculate_toxicity_scores(credentials, input_path, output_path):
    services = [
        discovery.build(
            "commentanalyzer",
            "v1alpha1",
            developerKey=value,
            discoveryServiceUrl="https://commentanalyzer.googleapis.com/$discovery/rest?version=v1alpha1",
            static_discovery=False,
        ) for _, value in credentials.items()
    ]

    n_services = len(services)
    filenames = [file.split("/")[-1] for file in glob.glob(os.path.join(input_path, "*"))]
    
    
    for filename in tqdm(filenames, total=len(filenames)):
        
        sentence = open(os.path.join(input_path, filename)).read()
        df_sentences = pd.DataFrame().append({'text':sentence},ignore_index=True)
        
        try:
            for i, row in df_sentences.iterrows():
                df_sentences.loc[i, "score"] = get_toxicity_score(
                    services[i % n_services],
                    row.text,
                    "TOXICITY"
                )

                if (i % n_services) == 0:
                    time.sleep(.3)
        except Exception as e:
            # print(e)
            print(filename)

        df_sentences.to_csv(os.path.join(output_path, filename), index=False)

## Running Perspective

In [26]:
#INTERVIEW
calculate_toxicity_scores(
    credentials,
    'data/coraal/INTERVIEW/00_sampled/',
    'data/coraal/INTERVIEW/01_scored/'
)

 14%|███████████████████▏                                                                                                                 | 996/6885 [10:07<55:40,  1.76it/s]ERROR:root:The following error occured: 
({'vary': 'Origin, X-Origin, Referer', 'content-type': 'application/json; charset=UTF-8', 'date': 'Tue, 10 May 2022 20:07:31 GMT', 'server': 'ESF', 'cache-control': 'private', 'x-xss-protection': '0', 'x-frame-options': 'SAMEORIGIN', 'x-content-type-options': 'nosniff', 'alt-svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000,h3-Q050=":443"; ma=2592000,h3-Q046=":443"; ma=2592000,h3-Q043=":443"; ma=2592000,quic=":443"; ma=2592000; v="46,43"', 'transfer-encoding': 'chunked', 'status': '400', 'content-length': '492', '-content-encoding': 'gzip'}, b'{\n  "error": {\n    "code": 400,\n    "message": "Attribute TOXICITY does not support request languages: ja-Latn",\n    "status": "INVALID_ARGUMENT",\n    "details": [\n      {\n        "@type": "type.googleapis.com/google.commenta

In [27]:
#CORAAL
calculate_toxicity_scores(
    credentials,
    'data/coraal/CORAAL/00_sampled/',
    'data/coraal/CORAAL/01_scored/'
)

  2%|██                                                                                                                                 | 106/6885 [01:41<1:35:34,  1.18it/s]ERROR:root:The following error occured: 
({'vary': 'Origin, X-Origin, Referer', 'content-type': 'application/json; charset=UTF-8', 'date': 'Tue, 10 May 2022 21:06:16 GMT', 'server': 'ESF', 'cache-control': 'private', 'x-xss-protection': '0', 'x-frame-options': 'SAMEORIGIN', 'x-content-type-options': 'nosniff', 'alt-svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000,h3-Q050=":443"; ma=2592000,h3-Q046=":443"; ma=2592000,h3-Q043=":443"; ma=2592000,quic=":443"; ma=2592000; v="46,43"', 'transfer-encoding': 'chunked', 'status': '400', 'content-length': '482', '-content-encoding': 'gzip'}, b'{\n  "error": {\n    "code": 400,\n    "message": "Attribute TOXICITY does not support request languages: ro",\n    "status": "INVALID_ARGUMENT",\n    "details": [\n      {\n        "@type": "type.googleapis.com/google.commentanalyz

## Reading data

In [13]:
pd.DataFrame({
        'i': i,
        'CORAAL.File': data['CORAAL.File'],
        'CORAAL.Spkr': data['CORAAL.Spkr'],
        'sentences': 'sentences'
    })

,i,CORAAL.File,CORAAL.Spkr,sentences
0,2,VLD_se0_ag2_f_01_1,VLD_se0_ag2_f_01,sentences
1,2,VLD_se0_ag2_m_01_1,VLD_se0_ag2_m_01,sentences
2,2,VLD_se0_ag3_f_01_1,VLD_se0_ag3_f_01,sentences
3,2,VLD_se0_ag3_f_01_2,VLD_se0_ag3_f_01,sentences
4,2,VLD_se0_ag3_f_02_1,VLD_se0_ag3_f_02,sentences
5,2,VLD_se0_ag3_m_01_1,VLD_se0_ag3_m_01,sentences
6,2,VLD_se0_ag3_m_01_2,VLD_se0_ag3_m_01,sentences
7,2,VLD_se0_ag3_m_02_1,VLD_se0_ag3_m_02,sentences
8,2,VLD_se0_ag3_m_03_1,VLD_se0_ag3_m_03,sentences
9,2,VLD_se0_ag4_f_01_1,VLD_se0_ag4_f_01,sentences


In [94]:
sentences

['Ten, ten sixty-one.',
 'How many people you done did this? Take too long? Okay.',
 'Alright.',
 'Wow.',
 'Fifty-seven.',
 'L- uh, Valdosta.',
 'Nope.',
 'Hudson Docket.',
 'Mm-hm.',
 'South side.',
 'Off and on? Mm.',
 "It's hard to say, cause I've been traveling a lot.",
 'And shoot, I stay maybe sometime maybe five year, ten year leave, come back.',
 'Uh.',
 'Uh.',
 'Yeah.',
 'Most of my life.',
 'I lived in Maryl- Laurel, Maryland.',
 'Mm-hm.',
 "I've lived in Orlando.",
 'I lived in Tampa.',
 'St.',
 'Petersburg.',
 'Hm.',
 'Florida.',
 'Virginia Beach, /Virgingia/.',
 'No.',
 'Yeah.',
 'W- And New York.',
 'Uh-huh.',
 '/RD-SCHOOL-4/.',
 'Marching cat.',
 'Trumpet and tuba.',
 'Um.',
 "I'm disabled.",
 'Yeah.',
 'Here.',
 'Valdosta.',
 'Mm-hm.',
 'Hm? Uh-huh.',
 'Uh.',
 "He's disabled too now, but he used to mix paint.",
 'Housewife.',
 'No.',
 'Well, uh.',
 'W- Actually, when I was born, I was born in Hudson Docket.',
 '/Well man/, my parents lost some- lost a kid.',
 'So, my mo